# NBA Prediction Models
## Kevin Yang, Eric Lee, Derek Young

Our main goal is to build prediction models for NBA team performance. As a first step, we will scrape data from `stats.nba.com` and store relevant data in a local sqllite database.

First, we will test our scraping functionality on a simple player example.

In [8]:
import requests
import sqlite3
import pandas as pd

# Use svg backend for better quality
import matplotlib
matplotlib.use("svg")
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
matplotlib.rcParams['figure.figsize'] = (10.0, 5.0)

In [20]:
def get_player_gamelogs(player_id, season):
    """ Given a player id (int or string) and a season (string, ex: 2016-17), returns a (header, log_list) where the
        header represents a key describing the format of a log in log list
    Input:
        player_id (int or string): player ID number
        season (str): season string, ex: 2016-17
    Output:
        (header, log_list): header is a key describing the format of a log in log list
    """
    
    player_url = ("http://stats.nba.com/stats/playergamelog?DateFrom=&DateTo=&LeagueID=00&PlayerID=" 
                  + str(player_id) + "&Season=" + season + "&SeasonType=Regular+Season")
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.10; rv:39.0) Gecko/20100101 Firefox/39.0'}
    # request the URL and parse the JSON
    response = requests.get(player_url, headers = headers)
    response.raise_for_status() # raise exception if invalid response
    response_json = response.json()
    log_list = response_json['resultSets'][0]['rowSet']
    header = response_json['resultSets'][0]['headers']
    
    return (header, log_list)

# jeremy lin
player_id = 202391
season = "2016-17"

(header, plog_list) = get_player_gamelogs(player_id, season)  

print "header", header
print "player gamelogs", plog_list

header [u'SEASON_ID', u'Player_ID', u'Game_ID', u'GAME_DATE', u'MATCHUP', u'WL', u'MIN', u'FGM', u'FGA', u'FG_PCT', u'FG3M', u'FG3A', u'FG3_PCT', u'FTM', u'FTA', u'FT_PCT', u'OREB', u'DREB', u'REB', u'AST', u'STL', u'BLK', u'TOV', u'PF', u'PTS', u'PLUS_MINUS', u'VIDEO_AVAILABLE']
player gamelogs [[u'22016', 202391, u'0021600057', u'NOV 02, 2016', u'BKN vs. DET', u'W', 16, 4, 6, 0.667, 0, 1, 0.0, 2, 2, 1.0, 0, 2, 2, 4, 0, 0, 1, 0, 10, 8, 1], [u'22016', 202391, u'0021600043', u'OCT 31, 2016', u'BKN vs. CHI', u'L', 24, 6, 10, 0.6, 1, 2, 0.5, 1, 2, 0.5, 0, 2, 2, 4, 4, 0, 3, 0, 14, -15, 1], [u'22016', 202391, u'0021600031', u'OCT 29, 2016', u'BKN @ MIL', u'L', 33, 4, 16, 0.25, 1, 7, 0.143, 3, 5, 0.6, 1, 2, 3, 11, 3, 1, 4, 0, 12, 12, 2], [u'22016', 202391, u'0021600019', u'OCT 28, 2016', u'BKN vs. IND', u'W', 34, 6, 12, 0.5, 2, 4, 0.5, 7, 8, 0.875, 0, 9, 9, 9, 0, 1, 2, 1, 21, 0, 2], [u'22016', 202391, u'0021600006', u'OCT 26, 2016', u'BKN @ BOS', u'L', 28, 6, 14, 0.429, 2, 5, 0.4, 4, 5, 0.8,

Now we have a way to extract data for a given player and season. Next, we wish to find team specific data. Team specific data is important because we need to extract our features from these data points. These features, along with some player-specfic features, will be used in a supervised learning model to predict the result of an NBA game.

In [38]:
def get_league_gamelogs(season):
    """ Given a season (string, ex: 2016-17), returns a (header, log_list) where the
        header represents a key describing the format of a log in log list
    Input:
        season (str): season string, ex: 2016-17
    Output:
        (header, log_list): header is a key describing the format of a log in log list
    """
    league_log_url = ("http://stats.nba.com/stats/leaguegamelog?Counter=1000&DateFrom=&DateTo=&" + 
                  "Direction=DESC&LeagueID=00&PlayerOrTeam=T&Season=" + str(season) + 
                  "&SeasonType=Regular+Season&Sorter=PTS")
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.10; rv:39.0) Gecko/20100101 Firefox/39.0'}

    # request the URL and parse the JSON
    response = requests.get(league_log_url, headers = headers)
    response.raise_for_status() # raise exception if invalid response
    response_json = response.json()
    log_list = response_json['resultSets'][0]['rowSet']
    header = response_json['resultSets'][0]['headers']
    
    return (header, log_list)


# def check_table_exists(conn, table_name):
#     """ Check if a table table_name exists in the SQLite database given in conn
#     Input:
#         conn (sqlite3.Connection) : Connection object corresponding to the database; used to perform SQL commands.
#         table_name (string): Name of table to check
#     Output:
#         bool: Whether or not the table table_name exists in conn
#     """
    
#     cursor = conn.cursor()
#     query = "SELECT count(*) FROM sqlite_master WHERE type='table' AND name= + '" + table_name + "'"
    
#     cursor.execute(query)

def generate_year_list(start, yrs):
    """ Load nba gamelog data for the past yrs years as a games tables into an SQLite database given in conn
    Input:
        conn (sqlite3.Connection) : Connection object corresponding to the database; used to perform SQL commands.
        yrs (int): Number of years to include in table
    Output:
        None
    """
    pass
    
def load_all_gamelogs(conn, yrs):
    """ Load nba gamelog data for the past yrs years as a games tables into an SQLite database given in conn
    Input:
        conn (sqlite3.Connection) : Connection object corresponding to the database; used to perform SQL commands.
        yrs (int): Number of years to include in table
    Output:
        None
    """
    
    cursor = conn.cursor()
    
    # hardcode for now
    year_list = ['2010-11', '2011-12', '2012-13', '2013-14', '2015-16', '2016-17']
    
    # clear league_log table
    cursor.execute('drop table if exists league_log')
    
    # create big table
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS league_log (
    season_id TEXT, 
    team_id INTEGER,
    team_abbreviation TEXT,
    team_name TEXT,
    game_id INTEGER,
    game_date INTEGER,
    matchup INTEGER,
    wl STRING,
    min INTEGER,
    fgm INTEGER,
    fga INTEGER,
    fg_pct REAL,
    fg3m INTEGER,
    fg3a INTEGER,
    fg3_pct REAL,
    ftm INTEGER,
    fta INTEGER,
    ft_pct REAL,
    oreb INTEGER,
    dreb INTEGER,
    reb INTEGER,
    ast INTEGER,
    stl INTEGER,
    blk INTEGER,
    tov INTEGER,
    pf INTEGER,
    pts INTEGER,
    plus_minus INTEGER
    )""")
    
    for year in year_list:
        (header, log_list) = get_league_gamelogs(year)
        print header
        
        question_marks = "(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ? ,?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
        query_string = "INSERT INTO league_log VALUES " + question_marks
        for log in log_list:
            cursor.execute(query_string,
                          (log[0],log[1],log[2],log[3],log[4],log[5],log[6],log[7],
                          log[8], log[9], log[10], log[11], log[12], log[13], log[14],
                          log[15],log[16], log[17], log[18], log[19], log[20], log[21],
                          log[22], log[23], log[24], log[25], log[26], log[27]))
            
    conn.commit()

In [39]:
conn = sqlite3.connect(r"db/league.db")
conn.text_factory = str
load_all_gamelogs(conn, 0)

[u'SEASON_ID', u'TEAM_ID', u'TEAM_ABBREVIATION', u'TEAM_NAME', u'GAME_ID', u'GAME_DATE', u'MATCHUP', u'WL', u'MIN', u'FGM', u'FGA', u'FG_PCT', u'FG3M', u'FG3A', u'FG3_PCT', u'FTM', u'FTA', u'FT_PCT', u'OREB', u'DREB', u'REB', u'AST', u'STL', u'BLK', u'TOV', u'PF', u'PTS', u'PLUS_MINUS', u'VIDEO_AVAILABLE']
[u'SEASON_ID', u'TEAM_ID', u'TEAM_ABBREVIATION', u'TEAM_NAME', u'GAME_ID', u'GAME_DATE', u'MATCHUP', u'WL', u'MIN', u'FGM', u'FGA', u'FG_PCT', u'FG3M', u'FG3A', u'FG3_PCT', u'FTM', u'FTA', u'FT_PCT', u'OREB', u'DREB', u'REB', u'AST', u'STL', u'BLK', u'TOV', u'PF', u'PTS', u'PLUS_MINUS', u'VIDEO_AVAILABLE']
[u'SEASON_ID', u'TEAM_ID', u'TEAM_ABBREVIATION', u'TEAM_NAME', u'GAME_ID', u'GAME_DATE', u'MATCHUP', u'WL', u'MIN', u'FGM', u'FGA', u'FG_PCT', u'FG3M', u'FG3A', u'FG3_PCT', u'FTM', u'FTA', u'FT_PCT', u'OREB', u'DREB', u'REB', u'AST', u'STL', u'BLK', u'TOV', u'PF', u'PTS', u'PLUS_MINUS', u'VIDEO_AVAILABLE']
[u'SEASON_ID', u'TEAM_ID', u'TEAM_ABBREVIATION', u'TEAM_NAME', u'GAME_ID', u

In [40]:
df = pd.read_sql_query('SELECT * FROM league_log', conn)
df.head()

,season_id,team_id,team_abbreviation,team_name,game_id,game_date,matchup,wl,min,fgm,...,oreb,dreb,reb,ast,stl,blk,tov,pf,pts,plus_minus
0,22010,1610612754,IND,Indiana Pacers,21000101,2010-11-09,IND vs. DEN,W,240,56,...,10,35,45,37,7,7,14,20,144,31
1,22010,1610612747,LAL,Los Angeles Lakers,21001053,2011-03-22,LAL vs. PHX,W,315,53,...,18,38,56,30,10,5,9,18,139,2
2,22010,1610612744,GSW,Golden State Warriors,21001079,2011-03-25,GSW vs. TOR,W,240,51,...,7,33,40,33,12,5,10,22,138,38
3,22010,1610612743,DEN,Denver Nuggets,21000248,2010-11-28,DEN vs. PHX,W,240,43,...,15,34,49,28,5,7,10,26,138,5
4,22010,1610612756,PHX,Phoenix Suns,21001053,2011-03-22,PHX @ LAL,L,315,51,...,10,42,52,34,3,5,13,24,137,-2
